# Open Medic — Prescription Data

**What:** Annual datasets on medication dispensed in community pharmacies (2014-2024). Volumes, reimbursement amounts, by ATC class, CIP code, prescriber specialty, patient demographics, and geography.

**Why it matters:** The analytical core — prescription patterns by molecule, prescriber type, and territory.

| Property | Value |
|----------|-------|
| Source | Ameli / CNAM open data |
| Format | CSV, semicolon-delimited |
| Encoding | **Latin-1** (ISO-8859-1) — not UTF-8! |
| Size | ~50-200 MB per year |

In [ ]:
import httpx
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

RAW_DIR = Path("../../data/raw/open_medic")
RAW_DIR.mkdir(parents=True, exist_ok=True)

# URL pattern for Open Medic annual files
def open_medic_url(year: int) -> str:
    return f"https://open-data-assurance-maladie.ameli.fr/medicaments/download.php?Dir_Rep=Open_MEDIC_Base_Complete&Annee={year}"

In [ ]:
# Download just 2024 for exploration (most recent)
YEAR = 2024
dest = RAW_DIR / f"open_medic_{YEAR}.csv"

if not dest.exists():
    url = open_medic_url(YEAR)
    print(f"Downloading Open Medic {YEAR}...")
    print(f"URL: {url}")
    with httpx.stream("GET", url, follow_redirects=True, timeout=300) as resp:
        resp.raise_for_status()
        total = int(resp.headers.get("content-length", 0))
        downloaded = 0
        with open(dest, "wb") as f:
            for chunk in resp.iter_bytes(65536):
                f.write(chunk)
                downloaded += len(chunk)
                if total:
                    print(f"\r  {downloaded/1e6:.0f}/{total/1e6:.0f} MB ({100*downloaded/total:.0f}%)", end="")
    print(f"\nDone: {dest.stat().st_size/1e6:.0f} MB")
else:
    print(f"Already exists: {dest.stat().st_size/1e6:.0f} MB")

In [ ]:
# Load — note: Latin-1 encoding and semicolon separator!
df = pd.read_csv(dest, sep=";", encoding="latin-1", low_memory=False)
print(f"Loaded {len(df):,} rows")
print(f"\nColumns ({len(df.columns)}):\n{list(df.columns)}")
df.head()

In [ ]:
df.dtypes

In [ ]:
# Basic stats
print(f"Rows: {len(df):,}")
print(f"Unique CIP13 codes: {df['CIP13'].nunique():,}" if 'CIP13' in df.columns else "CIP13 column not found")

# Find numeric columns
num_cols = df.select_dtypes(include='number').columns.tolist()
print(f"\nNumeric columns: {num_cols}")
df[num_cols].describe()

In [ ]:
# ATC classification levels present
atc_cols = [c for c in df.columns if 'atc' in c.lower() or 'ATC' in c]
print(f"ATC columns: {atc_cols}")

for col in atc_cols:
    print(f"\n{col}: {df[col].nunique()} unique values")
    print(df[col].value_counts().head(10).to_string())

In [ ]:
# Find the boxes/volume column
box_col = [c for c in df.columns if 'boite' in c.lower() or 'boi' in c.lower() or 'nbc' in c.lower()]
rem_col = [c for c in df.columns if 'rem' in c.lower() or 'rembours' in c.lower()]
print(f"Box/volume columns: {box_col}")
print(f"Reimbursement columns: {rem_col}")

In [ ]:
# Top prescriber specialties
psp_col = [c for c in df.columns if 'psp' in c.lower() or 'prescri' in c.lower() or 'spe' in c.lower()]
print(f"Prescriber columns: {psp_col}")

if psp_col:
    col = psp_col[0]
    top_prescribers = df[col].value_counts().head(15)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    top_prescribers.sort_values().plot.barh(ax=ax, color="steelblue")
    ax.set_title(f"Top 15 Prescriber Categories ({YEAR})")
    ax.set_xlabel("Number of rows")
    plt.tight_layout()
    plt.show()

In [ ]:
# Top drugs by boxes dispensed
if box_col and 'CIP13' in df.columns:
    bc = box_col[0]
    top_drugs = df.groupby('CIP13')[bc].sum().sort_values(ascending=False).head(20)
    
    fig, ax = plt.subplots(figsize=(12, 7))
    top_drugs.sort_values().plot.barh(ax=ax, color="darkorange")
    ax.set_title(f"Top 20 Drugs by Boxes Dispensed ({YEAR})")
    ax.set_xlabel("Boxes dispensed")
    plt.tight_layout()
    plt.show()

In [ ]:
# Geographic distribution
geo_cols = [c for c in df.columns if 'dep' in c.lower() or 'reg' in c.lower() or 'geo' in c.lower()]
print(f"Geographic columns: {geo_cols}")

if geo_cols and box_col:
    gc = geo_cols[0]
    bc = box_col[0]
    by_region = df.groupby(gc)[bc].sum().sort_values(ascending=False).head(20)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    by_region.sort_values().plot.barh(ax=ax, color="seagreen")
    ax.set_title(f"Top 20 Territories by Boxes Dispensed ({YEAR})")
    ax.set_xlabel("Total boxes")
    plt.tight_layout()
    plt.show()

In [ ]:
# Age/sex breakdown
age_col = [c for c in df.columns if 'age' in c.lower()]
sex_col = [c for c in df.columns if 'sex' in c.lower()]
print(f"Age columns: {age_col}")
print(f"Sex columns: {sex_col}")

if age_col:
    print(f"\nAge groups: {sorted(df[age_col[0]].unique())}")
if sex_col:
    print(f"Sex values: {sorted(df[sex_col[0]].unique())}")

In [ ]:
# Null rates
null_pct = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
print("Null % per column:\n")
print(null_pct.to_string())

## Multi-year comparison (optional — download more years first)
To compare across years, download additional years and load them:

In [ ]:
# Uncomment to download all years:
# for year in range(2014, 2025):
#     dest = RAW_DIR / f"open_medic_{year}.csv"
#     if dest.exists():
#         continue
#     print(f"Downloading {year}...")
#     resp = httpx.get(open_medic_url(year), follow_redirects=True, timeout=300)
#     resp.raise_for_status()
#     dest.write_bytes(resp.content)
#     print(f"  {dest.stat().st_size/1e6:.0f} MB")